<a href="https://colab.research.google.com/github/o-sapohova/A-B-testing/blob/main/A_B_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Calculating statistical significance**


In [ ]:
from google.colab import auth, drive
from google.cloud import bigquery
import pandas as pd
import numpy as np
from scipy.stats import norm

In [ ]:
auth.authenticate_user()
auth.authenticate_user()
client = bigquery.Client(project="data-analytics-mate")

query = """
WITH
  session_info AS (
  SELECT
    s.date,
    s.ga_session_id,
    sp.country,
    sp.device,
    sp.continent,
    sp.channel,
    ab.test,
    ab.test_group
  FROM
    `DA.ab_test` ab
  JOIN
    `DA.session` s
  ON
    ab.ga_session_id = s.ga_session_id
  JOIN
    `DA.session_params` sp
  ON
    sp.ga_session_id = s.ga_session_id ),
  session_with_orders AS (
  SELECT
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    COUNT(DISTINCT o.ga_session_id) AS session_with_orders
  FROM
    `DA.order` o
  JOIN
    session_info
  ON
    o.ga_session_id = session_info.ga_session_id
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group ),
  events AS (
  SELECT
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    ep.event_name,
    COUNT(ep.ga_session_id) AS event_cnt
  FROM
    `DA.event_params` ep
  JOIN
    session_info
  ON
    ep.ga_session_id = session_info.ga_session_id
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    ep.event_name ),
  session AS (
  SELECT
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    COUNT(DISTINCT session_info.ga_session_id) AS session_cnt
  FROM
    session_info
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group ),
  account AS (
  SELECT
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    COUNT(DISTINCT acs.ga_session_id) AS new_account_cnt
  FROM
    `DA.account_session` acs
  JOIN
    session_info
  ON
    acs.ga_session_id = session_info.ga_session_id
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group )
SELECT
  session_with_orders.date,
  session_with_orders.country,
  session_with_orders.device,
  session_with_orders.continent,
  session_with_orders.channel,
  session_with_orders.test,
  session_with_orders.test_group,
  'session_with_orders' AS event_name,
  session_with_orders.session_with_orders AS value
FROM
  session_with_orders
UNION ALL
SELECT
  events.date,
  events.country,
  events.device,
  events.continent,
  events.channel,
  events.test,
  events.test_group,
  events.event_name,
  events.event_cnt AS value
FROM
  events
UNION ALL
SELECT
  session.date,
  session.country,
  session.device,
  session.continent,
  session.channel,
  session.test,
  session.test_group,
  'session' AS event_name,
  session.session_cnt AS value
FROM
  session
UNION ALL
SELECT
  account.date,
  account.country,
  account.device,
  account.continent,
  account.channel,
  account.test,
  account.test_group,
  'new_account' AS event_name,
  account.new_account_cnt AS value
FROM
  account
"""

query_job = client.query(query)
results = query_job.result()

In [ ]:
df = results.to_dataframe()
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-12-08,Palestine,desktop,Asia,Direct,4,2,new_account,1
1,2020-12-08,Palestine,desktop,Asia,Direct,3,2,new_account,1
2,2020-11-06,Puerto Rico,desktop,Americas,Social Search,2,2,new_account,1
3,2020-11-06,Puerto Rico,desktop,Americas,Social Search,1,1,new_account,1
4,2020-12-08,Croatia,desktop,Europe,Direct,4,2,new_account,1


In [ ]:
# Finding a unique event_name to create a dict with metrics
df["event_name"].unique()

array(['new_account', 'session_with_orders', 'user_engagement',
       'page_view', 'session_start', 'first_visit', 'view_item', 'scroll',
       'view_promotion', 'begin_checkout', 'add_to_cart',
       'select_promotion', 'view_search_results', 'select_item',
       'add_shipping_info', 'add_payment_info', 'click', 'session',
       'view_item_list'], dtype=object)

In [ ]:
# Creating a dict with metrics
metrics = {
           #'metric': ('numerator_event', 'denominator_event'),
           'add_to_cart/session': ('add_to_cart', 'session'),
           'begin_checkout/session': ('begin_checkout', 'session'),
           'add_shipping_info/session': ('add_shipping_info', 'session'),
           'add_payment_info/session': ('add_payment_info', 'session'),
           'new_account/session': ('new_account', 'session'),
           'session_with_orders/session': ('session_with_orders', 'session')
           }

## Total

In [ ]:
# Calculating statistical significance for metrics by test number
results = []
tests = df["test"].astype(int).unique().tolist()
for test in tests:
    df_test = df[df["test"] == test]
    # Calculating metric_change, z-stat and p-value for each metric in metrics
    for metric, (num_event, den_event) in metrics.items():
        num_count_A = df_test[(df_test["event_name"] == num_event) & (df_test["test_group"] == 1)]['value'].sum()
        den_count_A = df_test[(df_test["event_name"] == den_event) & (df_test["test_group"] == 1)]['value'].sum()
        conversion_rate_A = num_count_A / den_count_A
        num_count_B = df_test[(df_test["event_name"] == num_event) & (df_test["test_group"] == 2)]['value'].sum()
        den_count_B = df_test[(df_test["event_name"] == den_event) & (df_test["test_group"] == 2)]['value'].sum()
        conversion_rate_B = num_count_B / den_count_B
        metric_change = (conversion_rate_B/conversion_rate_A - 1) * 100
        # z-test
        p_pool = (num_count_A + num_count_B) / (den_count_A + den_count_B)
        se = np.sqrt(p_pool * (1 - p_pool) * (1/den_count_A + 1/den_count_B))
        z_stat = (conversion_rate_B - conversion_rate_A) / se
        p_value = 2 * (1 - norm.cdf(abs(z_stat)))
        significant = p_value < 0.05


        results.append({
            'test_number': test,
            'metric': metric,
            'numerator_event': num_event,
            'denominator_event': den_event,
            'numerator_count_A': num_count_A,
            'denominator_count_A': den_count_A,
            'conversion_rate_A': conversion_rate_A,
            'numerator_count_B': num_count_B,
            'denominator_count_B': den_count_B,
            'conversion_rate_B': conversion_rate_B,
            'metric_change': metric_change,
            'z_stat': z_stat,
            'p_value': p_value,
            'significant': significant
        })

In [ ]:
results_df = pd.DataFrame(results)
display(results_df)

,test_number,metric,numerator_event,denominator_event,numerator_count_A,denominator_count_A,conversion_rate_A,numerator_count_B,denominator_count_B,conversion_rate_B,metric_change,z_stat,p_value,significant
0,4,add_to_cart/session,add_to_cart,session,21536,105079,0.204951,21554,105141,0.205001,0.024563,0.028589,0.977192,False
1,4,begin_checkout/session,begin_checkout,session,12555,105079,0.119482,12267,105141,0.116672,-2.351523,-1.995998,0.045934,True
2,4,add_shipping_info/session,add_shipping_info,session,5128,105079,0.048801,4956,105141,0.047137,-3.411125,-1.785795,0.074132,False
3,4,add_payment_info/session,add_payment_info,session,3731,105079,0.035507,3601,105141,0.034249,-3.541234,-1.571106,0.116158,False
4,4,new_account/session,new_account,session,8984,105079,0.085498,8687,105141,0.082622,-3.362896,-2.375457,0.017527,True
5,4,session_with_orders/session,session_with_orders,session,10596,105079,0.100838,10481,105141,0.099685,-1.143644,-0.880234,0.378732,False
6,3,add_to_cart/session,add_to_cart,session,17674,70047,0.252316,17229,70439,0.244595,-3.060321,-3.348897,0.000811,True
7,3,begin_checkout/session,begin_checkout,session,9532,70047,0.136080,9264,70439,0.131518,-3.352445,-2.511389,0.012026,True
8,3,add_shipping_info/session,add_shipping_info,session,5298,70047,0.075635,5188,70439,0.073652,-2.621211,-1.413727,0.157442,False
9,3,add_payment_info/session,add_payment_info,session,3623,70047,0.051722,3697,70439,0.052485,1.474630,0.643172,0.520112,False


## by Continent

In [ ]:
# Calculating statistical significance for metrics by test number and continent
results_by_continent = []

tests = df["test"].astype(int).unique().tolist()
continents = df["continent"].unique().tolist()

for test in tests:
    df_test = df[df["test"] == test]
    for continent in continents:
        df_continent = df_test[df_test["continent"] == continent]
        # Calculating metric_change, z-stat and p-value for each metric in metrics
        for metric, (num_event, den_event) in metrics.items():
            num_count_A = df_continent[(df_continent["event_name"] == num_event) & (df_continent["test_group"] == 1)]['value'].sum()
            den_count_A = df_continent[(df_continent["event_name"] == den_event) & (df_continent["test_group"] == 1)]['value'].sum()
            conversion_rate_A = num_count_A / den_count_A
            num_count_B = df_continent[(df_continent["event_name"] == num_event) & (df_continent["test_group"] == 2)]['value'].sum()
            den_count_B = df_continent[(df_continent["event_name"] == den_event) & (df_continent["test_group"] == 2)]['value'].sum()
            conversion_rate_B = num_count_B / den_count_B
            metric_change = (conversion_rate_B/conversion_rate_A - 1) * 100
            # z-test
            p_pool = (num_count_A + num_count_B) / (den_count_A + den_count_B)
            se = np.sqrt(p_pool * (1 - p_pool) * (1/den_count_A + 1/den_count_B))
            z_stat = (conversion_rate_B - conversion_rate_A) / se
            p_value = 2 * (1 - norm.cdf(abs(z_stat)))
            significant = p_value < 0.05


            results_by_continent.append({
                'test_number': test,
                'continent': continent,
                'metric': metric,
                'numerator_event': num_event,
                'denominator_event': den_event,
                'numerator_count_A': num_count_A,
                'denominator_count_A': den_count_A,
                'conversion_rate_A': conversion_rate_A,
                'numerator_count_B': num_count_B,
                'denominator_count_B': den_count_B,
                'conversion_rate_B': conversion_rate_B,
                'metric_change': metric_change,
                'z_stat': z_stat,
                'p_value': p_value,
                'significant': significant
            })

In [ ]:
results_by_continent_df = pd.DataFrame(results_by_continent)
display(results_by_continent_df)

,test_number,continent,metric,numerator_event,denominator_event,numerator_count_A,denominator_count_A,conversion_rate_A,numerator_count_B,denominator_count_B,conversion_rate_B,metric_change,z_stat,p_value,significant
0,4,Asia,add_to_cart/session,add_to_cart,session,4804,24908,0.192870,4672,25165,0.185655,-3.740909,-2.060874,0.039315,True
1,4,Asia,begin_checkout/session,begin_checkout,session,2694,24908,0.108158,2764,25165,0.109835,1.550571,0.602092,0.547113,False
2,4,Asia,add_shipping_info/session,add_shipping_info,session,1098,24908,0.044082,1124,25165,0.044665,1.322499,0.316746,0.751436,False
3,4,Asia,add_payment_info/session,add_payment_info,session,819,24908,0.032881,830,25165,0.032982,0.308125,0.063519,0.949354,False
4,4,Asia,new_account/session,new_account,session,2164,24908,0.086880,2039,25165,0.081025,-6.738608,-2.362187,0.018167,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,1,(not set),begin_checkout/session,begin_checkout,session,4,97,0.041237,22,100,0.220000,433.500000,3.706053,0.000211,True
140,1,(not set),add_shipping_info/session,add_shipping_info,session,4,97,0.041237,13,100,0.130000,215.250000,2.218144,0.026545,True
141,1,(not set),add_payment_info/session,add_payment_info,session,7,97,0.072165,10,100,0.100000,38.571429,0.695585,0.486689,False
142,1,(not set),new_account/session,new_account,session,7,97,0.072165,11,100,0.110000,52.428571,0.921405,0.356839,False


## by Device

In [ ]:
# Calculating statistical significance for metrics by test number and device
results_by_device = []

tests = df["test"].astype(int).unique().tolist()
devices = df["device"].unique().tolist()

for test in tests:
    df_test = df[df["test"] == test]
    for device in devices:
        df_device = df_test[df_test["device"] == device]
        # Calculating metric_change, z-stat and p-value for each metric in metrics
        for metric, (num_event, den_event) in metrics.items():
            num_count_A = df_device[(df_device["event_name"] == num_event) & (df_device["test_group"] == 1)]['value'].sum()
            den_count_A = df_device[(df_device["event_name"] == den_event) & (df_device["test_group"] == 1)]['value'].sum()
            conversion_rate_A = num_count_A / den_count_A
            num_count_B = df_device[(df_device["event_name"] == num_event) & (df_device["test_group"] == 2)]['value'].sum()
            den_count_B = df_device[(df_device["event_name"] == den_event) & (df_device["test_group"] == 2)]['value'].sum()
            conversion_rate_B = num_count_B / den_count_B
            metric_change = (conversion_rate_B/conversion_rate_A - 1) * 100
            # z-test
            p_pool = (num_count_A + num_count_B) / (den_count_A + den_count_B)
            se = np.sqrt(p_pool * (1 - p_pool) * (1/den_count_A + 1/den_count_B))
            z_stat = (conversion_rate_B - conversion_rate_A) / se
            p_value = 2 * (1 - norm.cdf(abs(z_stat)))
            significant = p_value < 0.05


            results_by_device.append({
                'test_number': test,
                'device': device,
                'metric': metric,
                'numerator_event': num_event,
                'denominator_event': den_event,
                'numerator_count_A': num_count_A,
                'denominator_count_A': den_count_A,
                'conversion_rate_A': conversion_rate_A,
                'numerator_count_B': num_count_B,
                'denominator_count_B': den_count_B,
                'conversion_rate_B': conversion_rate_B,
                'metric_change': metric_change,
                'z_stat': z_stat,
                'p_value': p_value,
                'significant': significant
            })

In [ ]:
results_by_device_df = pd.DataFrame(results_by_device)
display(results_by_device_df)

,test_number,device,metric,numerator_event,denominator_event,numerator_count_A,denominator_count_A,conversion_rate_A,numerator_count_B,denominator_count_B,conversion_rate_B,metric_change,z_stat,p_value,significant
0,4,desktop,add_to_cart/session,add_to_cart,session,12708,61484,0.206688,12959,61436,0.210935,2.054807,1.831685,0.066998,False
1,4,desktop,begin_checkout/session,begin_checkout,session,7456,61484,0.121267,7026,61436,0.114363,-5.693543,-3.754267,0.000174,True
2,4,desktop,add_shipping_info/session,add_shipping_info,session,3042,61484,0.049476,2845,61436,0.046308,-6.402932,-2.600630,0.009305,True
3,4,desktop,add_payment_info/session,add_payment_info,session,2211,61484,0.035961,2046,61436,0.033303,-7.390387,-2.547934,0.010836,True
4,4,desktop,new_account/session,new_account,session,5314,61484,0.086429,5105,61436,0.083095,-3.857950,-2.098595,0.035853,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,1,tablet,begin_checkout/session,begin_checkout,session,83,999,0.083083,56,1009,0.055500,-33.198801,-2.434631,0.014907,True
68,1,tablet,add_shipping_info/session,add_shipping_info,session,66,999,0.066066,49,1009,0.048563,-26.493378,-1.687725,0.091464,False
69,1,tablet,add_payment_info/session,add_payment_info,session,48,999,0.048048,31,1009,0.030723,-36.056739,-1.996608,0.045868,True
70,1,tablet,new_account/session,new_account,session,90,999,0.090090,93,1009,0.092170,2.309217,0.161955,0.871341,False


## by Channel

In [ ]:
# Calculating statistical significance for metrics by test number and channel
results_by_channel = []

tests = df["test"].astype(int).unique().tolist()
channels = df["channel"].unique().tolist()

for test in tests:
    df_test = df[df["test"] == test]
    for channel in channels:
        df_channel = df_test[df_test["channel"] == channel]
        # Calculating metric_change, z-stat and p-value for each metric in metrics
        for metric, (num_event, den_event) in metrics.items():
            num_count_A = df_channel[(df_channel["event_name"] == num_event) & (df_channel["test_group"] == 1)]['value'].sum()
            den_count_A = df_channel[(df_channel["event_name"] == den_event) & (df_channel["test_group"] == 1)]['value'].sum()
            conversion_rate_A = num_count_A / den_count_A
            num_count_B = df_channel[(df_channel["event_name"] == num_event) & (df_channel["test_group"] == 2)]['value'].sum()
            den_count_B = df_channel[(df_channel["event_name"] == den_event) & (df_channel["test_group"] == 2)]['value'].sum()
            conversion_rate_B = num_count_B / den_count_B
            metric_change = (conversion_rate_B/conversion_rate_A - 1) * 100
            # z-test
            p_pool = (num_count_A + num_count_B) / (den_count_A + den_count_B)
            se = np.sqrt(p_pool * (1 - p_pool) * (1/den_count_A + 1/den_count_B))
            z_stat = (conversion_rate_B - conversion_rate_A) / se
            p_value = 2 * (1 - norm.cdf(abs(z_stat)))
            significant = p_value < 0.05


            results_by_channel.append({
                'test_number': test,
                'channel': channel,
                'metric': metric,
                'numerator_event': num_event,
                'denominator_event': den_event,
                'numerator_count_A': num_count_A,
                'denominator_count_A': den_count_A,
                'conversion_rate_A': conversion_rate_A,
                'numerator_count_B': num_count_B,
                'denominator_count_B': den_count_B,
                'conversion_rate_B': conversion_rate_B,
                'metric_change': metric_change,
                'z_stat': z_stat,
                'p_value': p_value,
                'significant': significant
            })

In [ ]:
results_by_channel_df = pd.DataFrame(results_by_channel)
display(results_by_channel_df)

,test_number,channel,metric,numerator_event,denominator_event,numerator_count_A,denominator_count_A,conversion_rate_A,numerator_count_B,denominator_count_B,conversion_rate_B,metric_change,z_stat,p_value,significant
0,4,Direct,add_to_cart/session,add_to_cart,session,4981,24395,0.204181,5192,24653,0.210603,3.145240,1.753903,7.944706e-02,False
1,4,Direct,begin_checkout/session,begin_checkout,session,2853,24395,0.116950,2826,24653,0.114631,-1.982994,-0.802588,4.222129e-01,False
2,4,Direct,add_shipping_info/session,add_shipping_info,session,1191,24395,0.048821,1150,24653,0.046647,-4.452985,-1.129188,2.588184e-01,False
3,4,Direct,add_payment_info/session,add_payment_info,session,893,24395,0.036606,831,24653,0.033708,-7.916756,-1.742547,8.141279e-02,False
4,4,Direct,new_account/session,new_account,session,2102,24395,0.086165,1980,24653,0.080315,-6.789782,-2.345336,1.900995e-02,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,1,Undefined,begin_checkout/session,begin_checkout,session,353,3336,0.105815,569,3397,0.167501,58.295313,7.361334,1.820766e-13,True
116,1,Undefined,add_shipping_info/session,add_shipping_info,session,279,3336,0.083633,436,3397,0.128349,53.466215,5.954477,2.609056e-09,True
117,1,Undefined,add_payment_info/session,add_payment_info,session,269,3336,0.080635,416,3397,0.122461,51.869844,5.676166,1.377470e-08,True
118,1,Undefined,new_account/session,new_account,session,301,3336,0.090228,247,3397,0.072711,-19.413749,-2.628178,8.584350e-03,True


# **Conclusions**

## Test 1

1. ***add_payment_info/session:***
   * **total:** конверсія статистично значуще збільшилася на 12,54%;
   * ***by Continent:*** конверсія збільшилась на всіх континентах, але статистично значущі результати є лише у континента *Europe* (+40,3%);
   * ***by Device:*** статистично значуще збільшення для *Desktop* (+11,36%) та для *Mobil* (+17,14%), також статистично значуще зниження для *Tablet* (-36,06%);
   * ***by Channel:*** статистично значуще збільшення для *Direct* (+35,83%) та для *Paid Search* (+14,71%), також статистично значуще зниження для *Organic Search* (-19,46%).

2. ***add_shipping_info/session:***
   * **total:** конверсія статистично значуще збільшилася на 6,56%;
   * ***by Continent:*** конверсія збільшилась на всіх континентах (окрім статистично не значущого зменшення -28,8% для *Africa*), але статистично значущі результати є лише у континента *Europe* (+15,4%);
   * ***by Device:*** статистично значуще збільшення для *Desktop* (+12,2%);
   * ***by Channel:*** статистично значуще збільшення для *Direct* (+11,3%), також статистично значуще зниження для *Organic Search* (-9,4%).

3. ***begin_checkout/session:***
   * **total:** конверсія статистично значуще збільшилася на 6,66%;
   * ***by Continent:*** конверсія статистично значуще знизилась для континента *Africa* (-36,3%), але статистично значуще збільшилась для *Europe* (+16%) та *Oceania* (+59,96%);
   * ***by Device:*** статистично значуще збільшення для *Desktop* (+14,26%) та статистично значуще зниження для *Tablet* (-33,2%);
   * ***by Channel:*** статистично значуще збільшення для *Direct* (+14,72%), також статистично значуще зниження для *Organic Search* (-8,15%).

## Test 2

***add_to_cart/session:***
   * **total:** конверсія статистично значуще збільшилася на 9,75%;
   * ***by Continent:*** конверсія статистично збільшилась для континентів *Americas* (+25,2%) та *Oceania* (+528,3%), але статистично значуще знизилась *Asia* (-17,7%);
   * ***by Device:*** статистично значуще збільшення для *Desktop* (+12,83%);
   * ***by Channel:*** статистично значуще збільшення для *Paid Search* (+16,7%) та *Social Search* (+42,4%).

## Test 3

* **total:** всі метрики, які досліджувались, зменшились, статистично значуще знизились метрики **add_to_cart/session** (-3,06%) та **begin_checkout/session** (-3,35%).
* **by Continent:** немає статистично значущих позивних результатів.
* **by Device:** статистично значущій позитивний результат (+7,85%) для метрики **session_with_orders/session** для пристрою *Mobile*.
* **by Channel:** статистично значущій позитивний результат (+5,15%) для метрики **add_to_cart/session** для *Paid Search*.

## Test 4

* **total:** всі метрики, які досліджувались, зменшились, статистично значуще знизились **new_account/session** (-3,36%) та **begin_checkout/session** (-2,35%).
* **by Continent:**
   * статистично значущі позивні результати:
     * **add_to_cart/session:** *Africa* (+88,58%)
     * **begin_checkout/session:** *Africa* (+43,14%);
    * статистично значущі негативні результати:
      * **add_payment_info/session:** *Europe* (-11,7%), *Oceania* (-48.1%);
      *  **add_shipping_info/session:** *Europe* (-12,35%), *Oceania* (-33,6%);
      * **add_to_cart/session:** *Asia* (-3,74%), *Oceania* (-29,8%);
      * **begin_checkout/session:** *Oceania* (-35,6%);
      * **new_account/session:** *Asia* (-6,74%).
* **by Device:**
   * статистично значущій позитивний результат:
     *  **begin_checkout/session:** *Mobile* (+4,29%);
   * статистично значущі негативні результати:
     * **add_payment_info/session:** *Desktop* (-7,39%), *Tablet* (-49,1%);
     *  **add_shipping_info/session:**  *Desktop* (-6,4%), *Tablet* (-29,03%);
     * **add_to_cart/session:** *Tablet* (-20,21%);
     * **begin_checkout/session:** *Desktop* (-5,7%), *Tablet* (-24,8%);
     * **new_account/session:** *Desktop* (-3,86%);
     * **session_with_orders:** *Mobile* (-4,23%).

* **by Channel:**
  * статистично значущі позивні результати:
     * **add_to_cart/session:** *Paid Search* (+3,8%)
     * **begin_checkout/session:** *Paid Search* (+6,88%);
  * статистично значущі негативні результати:
      * **add_payment_info/session:** *Social Search* (-20,78%);
      *  **add_shipping_info/session:** *Organic Search* (-7,27%), *Social Search* (-10,78%);
      * **begin_checkout/session:** *Organic Search* (-3,78%), *Social Search* (-9,43%);
      * **new_account/session:** *Direct* (-6,79%), *Organic Search* (-5,06%).

# **Recommendations**

## Test 1
1. Запровадити зміни, які були протестовані, загалом, оскільки всі ключові метрики (**begin_checkout**, **add_shipping_info**, **add_payment_info**) показали статистично значуще зростання конверсії в цілому.
2. Провести аналіз причин негативної динаміки для:
  * *Tablet*, де зафіксовано значне статистично значуще падіння (-36,06% для **add_payment_info**, -33,2% для **begin_checkout**);
  * для трафіку з органічного пошуку *Organic Search* для всіх метрик;
  * *Africa* (-36,3% для **begin_checkout**), оскільки це може вказувати на локальні проблеми.
3. Запланувати наступні тести:
  * окремий A/B тест з виправленнями, спрямованими на покращення досвіду користувачів на *Tablet*, щоб мінімізувати виявлені втрати;
  * дослідити, що саме спричинило значне зростання конверсії для *Oceania* (+59,96% для **begin_checkout**), та спробувати масштабувати на інші регіони;
  * тестувати додаткові оптимізації, спрямовані на користувачів каналу *Direct*, оскільки вони найкраще реагують на зміни (стабільне зростання від +11% до +35%).


## Test 2
1. Запровадити зміни, які були протестовані, загалом, оскільки метрика **add_to_cart/session** статистично значуще зросла на +9,75% у тоталі.
2. Провести аналіз негативної динаміки для *Asia* на -17,7%.
3. Запланувати наступні тести:
  * дослідити причини дуже великого (+528,3%) зростання конверсії в *Oceania* і протестувати ці елементи/зміни на інших ринках;
  * для збільшення метрики для *Mobile* та *Tablet* пристроїв.


## Test 3
1. Негайно зупинити впровадження змін, оскільки вони призвели до статистично значущого зниження ключових метрик **add_to_cart/session** та **begin_checkout/session** у тоталі.
2. Провести поглиблений аналіз негативної динаміки в цілому та аналіз позитивних змін для:
  * виокремити зміни, які призвели до статистично значущого зростання метрики **session_with_orders/session** (+7,85%) на *Mobile*;
  * виокремити зміни, які покращили **add_to_cart/session** (+5,15%) для *Paid Search* трафіку.
3. Запланувати наступні тести:
  * повторні тести для успішних елементів окремо та у комбінації з виправленням виявлених проблем;
  * нові тести для збільшення метрик **add_to_cart/session** та **begin_checkout/session**, особливо приділити увагу сегментам з найбільшим падінням в даному тесті.

## Test 4
1. Зупинити впровадження змін, що тестувались, оскільки загальне статистично значуще зниження ключових метрик, таких як **new_account/session** (-3,36%) та **begin_checkout/session** (-2,35%), вказує на збитковість змін у їх поточному вигляді.
2. Провести аналіз причин негативних змін:
  * для *Tablet* зниження всіх метрик (наприклад, **add_payment_info/session** -49,1%);
  * причини падіння конверсії на *Desktop** (-5,7% для **begin_checkout/session** та -7,39% для **add_payment_info/session**);
  * причини значного падіння конверсії в *Oceania* та *Europe* на етапах оплати та доставки;
  * причини зниження конверсії на ключовихтканалах: *Organic Search* та *Social Search*.
3. Провести аналіз причин позитивних змін:
  * виокремити елементи, які призвели до зростання:
    * *Mobile* (+4,29% на **begin_checkout/session**);
    * *Paid Search* (зростання на **add_to_cart/session** та **begin_checkout/session**).
  * ретельно дослідити феноменальне зростання в *Africa* для **add_to_cart/session** (+88,58%) та **begin_checkout/session** (+43,14%).
4. Запланувати наступні тести для збільшення конверсіних метрик з урахуванням причин негативних та позитивних змін даного тесту.